In [1]:
!pip install sp3

  Using cached sp3-1.1.1-py3-none-any.whl


In [13]:
sat_list = {
    'TerraSAR-X': {
        'norad_id': 31698,
        'mass': 1230.0,          
        'cross_section': 4,  
        'cd': 2.2,             
        'cr': 1.5       
    },
    'CryoSat-2': {
        'norad_id': 36508,
        'mass': 720.0,          
        'cross_section': 4.6,  
        'cd': 2.2,             
        'cr': 1.5     
    },
    'Metop-2': {
        'norad_id': 29499,
        'mass': 4085.0,          
        'cross_section': 6.2,  
        'cd': 2.2,             
        'cr': 1.5     
    },
    'Beacon-C': {
        'norad_id': 1328,
        'mass': 60.8,          
        'cross_section': 0.45,  
        'cd': 2.2,             
        'cr': 1.5    
    },
    'DIADEME-1C': {
        'norad_id': 2674,
        'mass': None,          
        'cross_section': None,  
        'cd': 2.2,             
        'cr': 1.5    
    },
    'DIADEME-1D': {
        'norad_id': 2680,
        'mass': None,          
        'cross_section': None,  
        'cd': 2.2,             
        'cr': 1.5    
    },
    'IRS-P5 (CARTOSAT-1)': {
        'norad_id': 28649,
        'mass': 1560,          
        'cross_section': 2.7,  
        'cd': 2.2,             
        'cr': 1.5    
    },
    'TanDEM-X': {
        'norad_id': 31698,
        'mass': 1340,          
        'cross_section': 12,  
        'cd': 2.2,             
        'cr': 1.5   
    },
    'COSMIC-1 (FM-1)': {
        'norad_id': 29047,
        'mass': 70,          
        'cross_section': 2.5, #guesstimate based on images
        'cd': 2.2,             
        'cr': 1.5  
    },
    'COSMIC-1 (FM-2)': {
        'norad_id': 29048,
        'mass': 70,          
        'cross_section': 2.5, #guesstimate based on images
        'cd': 2.2,             
        'cr': 1.5  
    },
    'COSMIC-1 (FM-3)': {
        'norad_id': 29049,
        'mass': 70,          
        'cross_section': 2.5, #guesstimate based on images
        'cd': 2.2,             
        'cr': 1.5  
    },
    'COSMIC-1 (FM-4)': {
        'norad_id': 29050,
        'mass': 70,          
        'cross_section': 2.5, #guesstimate based on images
        'cd': 2.2,             
        'cr': 1.5  
    },
    'COSMIC-1 (FM-5)': {
        'norad_id': 29051,
        'mass': 70,          
        'cross_section': 2.5, #guesstimate based on images
        'cd': 2.2,             
        'cr': 1.5  
    },
    'COSMIC-1 (FM-6)': {
        'norad_id': 29052,
        'mass': 70,          
        'cross_section': 2.5, #guesstimate based on images
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Jason-2': {
        'norad_id': 33105,
        'mass': 550,          
        'cross_section': 5, #TODO: check with Marek
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Jason-3': {
        'norad_id': 41240,
        'mass': 553,          
        'cross_section': 5, #TODO: check with Marek
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Sentinel-6A/Jason-CS A': {
        'norad_id': 46984,
        'mass': 1200,          
        'cross_section': 5.13,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Metop-B': {
        'norad_id': 38771,
        'mass': 4087,          
        'cross_section': 17.6,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Metop-C': {
        'norad_id': 43689,
        'mass': 3950,          
        'cross_section': 17.6,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'HY-2A': {
        'norad_id': 37781,
        'mass': 1500,          
        'cross_section': 8.56,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'SARAL': {
        'norad_id': 39086,
        'mass': 407,          
        'cross_section': 1,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Swarm-A': {
        'norad_id': 39452,
        'mass': 468,          
        'cross_section': 9.1,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Swarm-B': {
        'norad_id': 39451,
        'mass': 468,          
        'cross_section': 9.1,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Swarm-C': {
        'norad_id': 39453,
        'mass': 468,          
        'cross_section': 9.1,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Ajisai': {
        'norad_id': 16908,
        'mass': 685,          
        'cross_section': 2.15,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Starlette': {
        'norad_id': 7646,
        'mass': 47.29,          
        'cross_section': 0.24,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Stella': {
        'norad_id': 22824,
        'mass': 48.0,          
        'cross_section': 0.24,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'Larets': {
        'norad_id': 27944,
        'mass': 21,          
        'cross_section': 0.21,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'LARES': {
        'norad_id': 38077,
        'mass': 386.8,          
        'cross_section': 0.182,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'CASSIOPE': {
        'norad_id': 39265,
        'mass': 500.0,          
        'cross_section': 1.8,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'GRACE-FO -A': {
        'norad_id': 43476,
        'mass': 600.2,          
        'cross_section': 3.123,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'GRACE-FO -B': {
        'norad_id': 43477,
        'mass': 600.2,          
        'cross_section': 3.123,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'PAZ': {
        'norad_id': 43215,
        'mass': 1350.0,          
        'cross_section': 5.0,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'HY-2B': {
        'norad_id': 43655,
        'mass': 1500.0,          
        'cross_section': 10.0,
        'cd': 2.2,             
        'cr': 1.5  
    },
    'HY-2C': {
        'norad_id': 46469,
        'mass': 1500.0,          
        'cross_section': 10.0,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'Sentinel-1A': {
        'norad_id': 39634,
        'mass': 2300.0,          
        'cross_section': 3.4,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'Sentinel-1B': {
        'norad_id': 41456,
        'mass': 2300.0,          
        'cross_section': 3.4,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'Sentinel-2A': {
        'norad_id': 40697,
        'mass': 1016.0,          
        'cross_section': 3.4,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'Sentinel-2B': {
        'norad_id': 42063,
        'mass': 1140.0,          
        'cross_section': 3.4,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'Sentinel-3A': {
        'norad_id': 41335,
        'mass': 1250.0,          
        'cross_section': 3.9,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'Sentinel-3B': {
        'norad_id': 43437,
        'mass': 1250.0,          
        'cross_section': 3.7,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'SWOT': {
        'norad_id': 54754,
        'mass': 2000.0,          
        'cross_section': 10.0,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'HY-2D': {
        'norad_id': 48621,
        'mass': 1500.0,          
        'cross_section': 10.0,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'COSMIC-2-1': {
        'norad_id': 44349,
        'mass': 300.0,          
        'cross_section': 1.25,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'COSMIC-2-2': {
        'norad_id': 44351,
        'mass': 300.0,          
        'cross_section': 1.25,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'COSMIC-2-3': {
        'norad_id': 44343,
        'mass': 300.0,          
        'cross_section': 1.25,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'COSMIC-2-4': {
        'norad_id': 44350,
        'mass': 300.0,          
        'cross_section': 1.25,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'COSMIC-2-5': {
        'norad_id': 44358,
        'mass': 300.0,          
        'cross_section': 1.25,
        'cd': 2.2,             
        'cr': 1.5 
    },
    'COSMIC-2-6': {
        'norad_id': 44353,
        'mass': 300.0,          
        'cross_section': 1.25,
        'cd': 2.2,             
        'cr': 1.5 
    }
}


In [11]:
# Define the additional values to add
additional_values = {
    'mass': 1000.0,          # kg; TODO: compute proper value
    'cross_section': 20.0,  # m^2; TODO: compute proper value
    'cd': 2.2,               # TODO: compute proper value
    'cr': 1.5                # TODO: compute proper value
}

# Iterate over each satellite in the dictionary and add the additional values
for satellite in sat_list.values():
    satellite.update(additional_values)

In [12]:
sat_list

{'TerraSAR-X': {'norad_id': 31698,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'Metop-2': {'norad_id': 29499,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'Beacon-C': {'norad_id': 1328,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'DIADEME-1C': {'norad_id': 2674,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'DIADEME-1D': {'norad_id': 2680,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'IRS-P5 (CARTOSAT-1)': {'norad_id': 28649,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'TanDEM-X': {'norad_id': 31698,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'COSMIC-1 (FM-1)': {'norad_id': 29047,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'COSMIC-1 (FM-2)': {'norad_id': 29048,
  'mass': 8000.0,
  'cross_section': 100.0,
  'cd': 2.0,
  'cr': 1.0},
 'COSMIC-1 (FM-3)': {'norad_id': 2904

In [9]:
import astropy.time
import astropy.coordinates
import matplotlib.pyplot as plt
import numpy as np
import sp3

# NORAD ID for GRACE FO satellite
satellite_id = sp3.NoradId("36508")

# Define the time range with a dense array of timestamps (one per minute)
start_time = astropy.time.Time("2021-01-01T00:00:00Z")
end_time = astropy.time.Time("2021-02-01T00:00:00Z")
time_points = astropy.time.Time(np.arange(start_time.jd, end_time.jd, 1/1440), format='jd')

# Download directory for SP3 data
download_directory = "sp3_cache"

# Get the interpolated ITRS positions for each minute
sp3_data = sp3.itrs(
    id=satellite_id,
    obstime=time_points,
    download_directory=download_directory,
    window=5,
    degree=10
)

# Calculate the norm of the position for each time point
norms = np.linalg.norm(sp3_data.cartesian.xyz.to_value(), axis=0)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(time_points.datetime, norms)
plt.xlabel('Time')
plt.ylabel('Norm of Position (m)')
plt.title('Norm of GRACE FO Position from 01/01/2021 to 02/01/2021')
plt.grid(True)
plt.show()


Exception: sp3.cddis.username and sp3.cddis.password are required to download CDDIS data, visit http://urs.earthdata.nasa.gov to register